In [1]:
import re
import time
from typing import List, Sequence, Tuple, Optional

import dimod
from dimod import ConstrainedQuadraticModel, Binary, cqm_to_bqm
from dwave.system import LeapHybridSampler


class GenCQMModel:
    """
    Model that converts CQM to BQM and solves using LeapHybridSampler (for BQM).
    
    Build flow:
      1. Build ConstrainedQuadraticModel (CQM)
      2. Convert to BQM via cqm_to_bqm()
      3. Solve with LeapHybridSampler (BQM hybrid)
    """
    
    def __init__(self, solver_name: Optional[str] = None, 
                 label: Optional[str] = None, 
                 bqm_lagrange: float = 10.0):
        self.cqm = ConstrainedQuadraticModel()
        self.rounds: List[dict] = []  # Each: {'in': [vars], 'out': [vars]}
        self.constraints = []         # Constraint labels for reference
        self.solver_name = solver_name
        self.label = label
        self._bqm_lagrange = bqm_lagrange
        
        self.sampleset = None
        self.best_feasible = None
        self.solve_times: List[float] = []
        self._objective_maximized = False

    # ========== Utility Methods ==========
    
    def _var_label(self, v) -> str:
        """Extract variable label (name)"""
        try:
            return next(iter(v.variables))
        except Exception:
            return getattr(v, 'name', str(v))

    def _var_value(self, sample: dict, v) -> int:
        """Extract variable value from sample"""
        return int(sample[self._var_label(v)])

    def _ensure_bqm(self, bqm) -> dimod.BinaryQuadraticModel:
        """Convert various forms of BQM objects to standard BinaryQuadraticModel"""
        if isinstance(bqm, dimod.BinaryQuadraticModel):
            return bqm
            
        # Tuple form: (linear, quadratic, offset)
        if isinstance(bqm, tuple) and len(bqm) == 3:
            lin, quad, off = bqm
            return dimod.BinaryQuadraticModel(lin, quad, off, vartype=dimod.BINARY)
        
        # Attribute access method
        try:
            lin = getattr(bqm, "linear")
            quad = getattr(bqm, "quadratic")
            off = getattr(bqm, "offset", 0.0)
            return dimod.BinaryQuadraticModel(lin, quad, off, vartype=dimod.BINARY)
        except Exception:
            pass
        
        # Use to_bqm() method
        if hasattr(bqm, "to_bqm"):
            return bqm.to_bqm()
            
        raise TypeError(f"Cannot convert to BinaryQuadraticModel: {type(bqm)}")

    def _get_bqm_sampler(self) -> LeapHybridSampler:
        """Return Hybrid Sampler for BQM"""
        try:
            if self.solver_name:
                return LeapHybridSampler(solver=self.solver_name)
            return LeapHybridSampler()
        except Exception:
            return LeapHybridSampler()

    def _pick_best_with_feasibility(self, sampleset):
        """Prioritize samples satisfying CQM constraints; return lowest energy sample if none"""
        try:
            feasibles = []
            for rec in sampleset.data():
                viol = self.cqm.check_feasible(rec.sample)
                
                # Handle based on check_feasible return type
                if isinstance(viol, dict):
                    ok = all(v == 0 for v in viol.values())
                elif isinstance(viol, (list, tuple)) and len(viol) > 0:
                    ok = bool(viol[0])
                else:
                    ok = bool(viol)
                    
                if ok:
                    feasibles.append(rec)
                    
            return feasibles[0] if feasibles else sampleset.first
        except Exception:
            return sampleset.first

    # ========== Model Building ==========
    
    def add_round(self, sbox_size: int, n_sboxes: int, 
                  prefix: str = "x") -> Tuple[Sequence[Binary], Sequence[Binary]]:
        """Add round: create input/output variables"""
        round_idx = len(self.rounds)
        n_bits = sbox_size * n_sboxes
        
        S_in = [Binary(f"{prefix}_{round_idx}_in_{i}") for i in range(n_bits)]
        S_out = [Binary(f"{prefix}_{round_idx}_out_{i}") for i in range(n_bits)]
        
        self.rounds.append({'in': S_in, 'out': S_out})
        return S_in, S_out

    def connect_rounds_with_permutation(self, prev_out: Sequence[Binary], 
                                       next_in: Sequence[Binary], 
                                       perm: Sequence[int]):
        """Connect rounds with permutation: next_in[perm[i]] = prev_out[i]"""
        assert len(prev_out) == len(next_in) == len(perm)
        
        for i, j in enumerate(perm):
            lbl = f"perm_{len(self.constraints)}_{i}"
            ct = self.cqm.add_constraint(next_in[j] - prev_out[i] == 0, label=lbl)
            self.constraints.append(ct)

    def add_constraint(self, coeffs: Sequence[int], vars: Sequence[Binary], 
                      rhs: int, sense: str = '==', 
                      label: Optional[str] = None):
        """Add linear constraint: sum(coeffs[i] * vars[i]) (sense) rhs"""
        lhs = sum(c * v for c, v in zip(coeffs, vars))
        
        if sense == '==':
            ct = self.cqm.add_constraint(lhs == rhs, label=label)
        elif sense == '<=':
            ct = self.cqm.add_constraint(lhs <= rhs, label=label)
        elif sense == '>=':
            ct = self.cqm.add_constraint(lhs >= rhs, label=label)
        else:
            raise ValueError(f"Unsupported constraint sense: {sense}")
            
        self.constraints.append(ct)
        return ct

    def add_constraint_from_string(self, expr: str, 
                                   a_vars: Sequence[Binary], 
                                   b_vars: Sequence[Binary],
                                   label_prefix: Optional[str] = None):
        """
        Parse and add constraint from string format
        e.g.: "+ a[0] + 2*a[1] - b[2] == 1"
        """
        # Separate sense
        if '>=' in expr:
            lhs, rhs = expr.split(">=")
            sense = '>='
        elif '<=' in expr:
            lhs, rhs = expr.split("<=")
            sense = '<='
        elif '==' in expr:
            lhs, rhs = expr.split("==")
            sense = '=='
        else:
            raise ValueError("Constraint must contain >=, <=, or ==")

        rhs = int(rhs.strip())
        
        # Parse terms: coefficient*variable or variable only
        pattern = re.compile(r'([+-]?\s*\d*\s*\*\s*[ab]\[\d+\]|[+-]?\s*[ab]\[\d+\])')
        terms = pattern.findall(lhs)

        coeffs = []
        vars_ = []

        for term in terms:
            term = term.replace(" ", "")
            
            # Process coefficient
            if '*' in term:
                coef_str, var_str = term.split('*')
                if coef_str in ('', '+'):
                    coef = 1
                elif coef_str == '-':
                    coef = -1
                else:
                    coef = int(coef_str)
            else:
                coef = -1 if term.startswith('-') else 1
                var_str = term.lstrip('+-')

            # Extract variable index
            var_type = var_str[0]  # 'a' or 'b'
            index = int(var_str[2:-1])
            
            v = a_vars[index] if var_type == 'a' else b_vars[index]
            coeffs.append(coef)
            vars_.append(v)

        lbl = f"ct_{label_prefix or 'sbox'}_{len(self.constraints)}"
        return self.add_constraint(coeffs, vars_, rhs, sense, label=lbl)

    def add_sbox_layer(self, input_vars: Sequence[Binary], 
                      output_vars: Sequence[Binary],
                      sbox_constraints: Sequence[str], 
                      sbox_size: int, 
                      n_boxes: int,
                      label_prefix: str = "sbox"):
        """Add S-box layer constraints"""
        assert len(input_vars) == len(output_vars) == sbox_size * n_boxes
        
        for i in range(n_boxes):
            a_block = input_vars[i * sbox_size:(i + 1) * sbox_size]
            b_block = output_vars[i * sbox_size:(i + 1) * sbox_size]
            
            for expr in sbox_constraints:
                self.add_constraint_from_string(expr, a_block, b_block,
                                              label_prefix=f"{label_prefix}_{i}")

    # ========== Objective ==========
    
    def set_objective(self, coeffs: Sequence[int], vars: Sequence[Binary], 
                     sense: str = 'min'):
        """Set objective function"""
        expr = sum(c * v for c, v in zip(coeffs, vars))
        
        if sense == 'min':
            self.cqm.set_objective(expr)
            self._objective_maximized = False
        elif sense == 'max':
            # CQM only supports minimization -> convert maximization to minimizing -expr
            self.cqm.set_objective(-expr)
            self._objective_maximized = True
        else:
            raise ValueError(f"Unsupported objective sense: {sense}")

    # ========== Solve ==========
    
    def _convert_to_bqm(self, lagrange: Optional[float] = None) -> Tuple[dimod.BinaryQuadraticModel, Optional[dict]]:
        """Convert CQM to BQM"""
        if lagrange is None:
            lagrange = self._bqm_lagrange
            
        # Handle return value based on dimod version
        try:
            out = cqm_to_bqm(self.cqm, lagrange_multiplier=lagrange, return_mapping=True)
        except TypeError:
            out = cqm_to_bqm(self.cqm, lagrange_multiplier=lagrange)

        if isinstance(out, tuple):
            raw_bqm = out[0]
            mapping = out[1] if len(out) >= 2 else None
        else:
            raw_bqm = out
            mapping = None

        bqm = self._ensure_bqm(raw_bqm)
        return bqm, mapping

    def solve(self, time_limit: Optional[int] = None, 
              label: Optional[str] = None, 
              lagrange: Optional[float] = None):
        """
        Convert CQM to BQM and solve with LeapHybridSampler
        
        Args:
            time_limit: Time limit (seconds)
            label: Job label
            lagrange: Lagrange multiplier for BQM conversion
        """
        t0 = time.time()
        
        # CQM -> BQM conversion
        bqm, _ = self._convert_to_bqm(lagrange=lagrange)
        
        # Solve with BQM sampler
        sampler = self._get_bqm_sampler()
        self.sampleset = sampler.sample(bqm, label=label or self.label, time_limit=time_limit)
        
        elapsed = time.time() - t0
        self.solve_times.append(elapsed)

        # Prioritize feasible samples
        try:
            feasible = self.sampleset.filter(lambda d: d.is_feasible)
        except Exception:
            feasible = None
            
        if feasible and len(feasible):
            self.best_feasible = feasible.first
        else:
            self.best_feasible = self._pick_best_with_feasibility(self.sampleset)
            
        return self.best_feasible

    # ========== Integral Distinguisher ==========
    
    def solve_for_integral_distinguisher(self, target_vars: Sequence[Binary], 
                                        blocksize: int = 64,
                                        verbose: bool = True, 
                                        time_limit: Optional[int] = None):
        """
        Integral distinguisher search loop
        
        Prerequisite: set_objective must be called with Minimize(sum(target_vars))
        
        Logic:
          - Find solution
          - objective > 1 (or multiple 1-bits) -> integral found
          - Otherwise, fix one 1-bit to 0 and repeat
        """
        counter = 0
        set_zero = []
        found = False
        integral_bits = []
        self.solve_times.clear()

        while counter < blocksize:
            row = self.solve(time_limit=time_limit)
            energy = row.energy
            obj_val = -energy if self._objective_maximized else energy
            
            if verbose:
                print(f"[+] Solve #{counter} -> Objective: {obj_val:.2f}")

            sample = row.sample
            ones_now = [i for i, v in enumerate(target_vars) 
                       if self._var_value(sample, v) == 1]

            # Termination condition: objective > 1 or 2+ 1-bits
            if obj_val > 1 or len(ones_now) > 1:
                found = True
                integral_bits = ones_now
                break

            # Force one 1-bit to 0
            progressed = False
            for v in target_vars:
                if self._var_value(sample, v) == 1:
                    lbl = f"force_zero_{self._var_label(v)}_{len(set_zero)}"
                    self.cqm.add_constraint(v == 0, label=lbl)
                    set_zero.append(self._var_label(v))
                    
                    if verbose:
                        print(f"  └─ Forcing {self._var_label(v)} = 0")
                    
                    counter += 1
                    progressed = True
                    break

            if not progressed:
                if verbose:
                    print("[X] No 1-bits to force; trivial or infeasible case.")
                found = True
                break

        # Print results
        print()
        if found:
            print("[✓] Integral Distinguisher FOUND!")
            if integral_bits:
                print(f"Integral bits (indices): {integral_bits}")
        else:
            print("[✗] No integral distinguisher exists.")

        if verbose:
            print("\nZero-forced bits:")
            if set_zero:
                for name in set_zero:
                    print(f"  - {name}")
            elif counter == 0:
                print("  (none — distinguisher confirmed without forcing)")
            else:
                print("  (none — infeasible from start)")

    # ========== Solution Readout ==========
    
    def get_solution(self) -> List[dict]:
        """Return input/output bit values for all rounds"""
        if self.best_feasible is None:
            raise RuntimeError("Call solve() first.")
            
        sample = self.best_feasible.sample
        sol = []
        
        for round_data in self.rounds:
            sol.append({
                key: [self._var_value(sample, v) for v in round_data[key]]
                for key in ('in', 'out')
            })
        return sol

    def print_solution(self):
        """Print current solution"""
        if self.best_feasible is None:
            raise RuntimeError("Call solve() first.")
            
        row = self.best_feasible
        obj = -row.energy if self._objective_maximized else row.energy
        
        print(f"Objective value: {obj}")
        for r, round_data in enumerate(self.rounds):
            for key in ('in', 'out'):
                bits = [self._var_value(row.sample, v) for v in round_data[key]]
                print(f"Round {r} {key}: {bits}")

In [2]:

# GIFT S-box
GIFT_SBOX = [1, 10, 4, 12, 6, 15, 3, 9, 2, 13, 11, 7, 5, 0, 8, 14]
# GIFT-16/32/64/128 Permutations, MSB, Source to Target representation
GIFT16_BITPERM = (12, 1, 6, 11, 8, 13, 2, 7, 4, 9, 14, 3, 0, 5, 10, 15)
GIFT32_BITPERM = (12, 1, 6, 11, 28, 17, 22, 27, 8, 13, 2, 7, 24, 29, 18, 23, 4, 9, 14, 3, 20, 25, 30, 19, 0, 5, 10, 15, 16, 21, 26, 31)
GIFT64_BITPERM = (12, 1, 6, 11, 28, 17, 22, 27, 44, 33, 38, 43, 60, 49, 54, 59, 8, 13, 2, 7, 24, 29, 18, 23, 40, 45, 34, 39, 56, 61, 50, 55, 4, 9, 14, 3, 20, 25, 30, 19, 36, 41, 46, 35, 52, 57, 62, 51, 0, 5, 10, 15, 16, 21, 26, 31, 32, 37, 42, 47, 48, 53, 58, 63)
GIFT128_BITPERM = (12, 1, 6, 11, 28, 17, 22, 27, 44, 33, 38, 43, 60, 49, 54, 59, 76, 65, 70, 75, 92, 81, 86, 91, 108, 97, 102, 107, 124, 113, 118, 123, 8, 13, 2, 7, 24, 29, 18, 23, 40, 45, 34, 39, 56, 61, 50, 55, 72, 77, 66, 71, 88, 93, 82, 87, 104, 109, 98, 103, 120, 125, 114, 119, 4, 9, 14, 3, 20, 25, 30, 19, 36, 41, 46, 35, 52, 57, 62, 51, 68, 73, 78, 67, 84, 89, 94, 83, 100, 105, 110, 99, 116, 121, 126, 115, 0, 5, 10, 15, 16, 21, 26, 31, 32, 37, 42, 47, 48, 53, 58, 63, 64, 69, 74, 79, 80, 85, 90, 95, 96, 101, 106, 111, 112, 117, 122, 127)

sbox_constraints = [
' a[0] + a[1] + a[2] + a[3] - b[0] - b[1] - b[2] - b[3] >= 0',
' - 3*a[0] - 3*a[1] - 5*a[2] - 4*a[3] + 2*b[0] + 3*b[1] + b[2] + b[3] >= -8',
' - 3*a[0] - 2*a[1] + 3*a[2] - a[3] - b[0] - 2*b[1] - 4*b[2] + 3*b[3] >= -7',
' - a[0] - a[1] - a[2] + 2*b[0] + 3*b[1] + b[2] + b[3] >= 0',
' 3*a[3] - b[0] - 2*b[1] - b[2] - b[3] >= -2',
' - a[1] - 2*a[3] - b[0] + b[1] + 2*b[2] - 2*b[3] >= -4',
' a[0] - a[3] + b[0] - b[1] - 2*b[2] - b[3] >= -3',
' - 3*a[0] - a[1] - 5*a[2] - 6*a[3] + 2*b[0] + b[1] + 5*b[2] + 3*b[3] >= -8',
' a[1] + 3*a[2] + a[3] - 2*b[0] - 2*b[1] - b[2] - 2*b[3] >= -2',
' a[1] + 3*a[3] - 2*b[0] - 2*b[1] - b[2] - b[3] >= -2',
' - a[0] - a[1] - a[3] + 3*b[0] + 2*b[1] + 2*b[2] + b[3] >= 0',
' - a[1] - a[3] - b[1] + b[2] + b[3] >= -2',
' 2*a[0] + a[1] + a[3] - b[0] - 2*b[1] - b[2] - 2*b[3] >= -2',
' - 2*a[1] - 2*a[2] - a[3] + b[0] + 2*b[1] + b[2] + b[3] >= -2',
' - a[0] - 2*a[3] - b[0] + b[1] - 2*b[2] + 2*b[3] >= -4']

Hbqm_v2p_16_time = []
Hbqm_v2p_32_time = []
Hbqm_v2p_64_time = []
Hbqm_v2p_128_time = []

Advantage2_1_6_16_time = []
Advantage2_1_6_32_time = []
Advantage2_1_6_64_time = []
Advantage2_1_6_128_time = []

Advantage_6_4_16_time = []
Advantage_6_4_32_time = []
Advantage_6_4_64_time = []
Advantage_6_4_128_time = []



# hybrid_binary_quadratic_model_version2p

# Application to GIFT-16

In [3]:
for idx in range(10):
    model = GenCQMModel(
        solver_name="hybrid_binary_quadratic_model_version2p",
        label="gift16-cqm2bqm",
        bqm_lagrange=15.0
    )

    SBOX_SIZE = 4
    N_SBOXES = 4
    BLOCKSIZE = SBOX_SIZE * N_SBOXES
    BITPERM = GIFT16_BITPERM
    assert len(BITPERM) == BLOCKSIZE
    assert set(BITPERM) == set(range(BLOCKSIZE))

    rounds = []
    n_rounds = 5

    for r in range(n_rounds):
        S_in, S_out = model.add_round(sbox_size=SBOX_SIZE, n_sboxes=N_SBOXES)
        rounds.append((S_in, S_out))
        model.add_sbox_layer(S_in, S_out, sbox_constraints,
                             sbox_size=SBOX_SIZE, n_boxes=N_SBOXES)

    for r in range(n_rounds - 1):
        model.connect_rounds_with_permutation(rounds[r][1], rounds[r + 1][0], BITPERM)

    for i in range(BLOCKSIZE):
        model.cqm.add_constraint(rounds[0][0][i] == (1 if i >= 1 else 0), label=f"fix_in_{i}")

    model.set_objective([1] * BLOCKSIZE, rounds[n_rounds - 1][1], sense='min')

    model.solve_for_integral_distinguisher(
        rounds[n_rounds - 1][1],
        blocksize=BLOCKSIZE,
        verbose=True
    )
    Hbqm_v2p_16_time.append(sum(model.solve_times))
print(Hbqm_v2p_16_time)

[+] Solve #0 -> Objective: 530.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [0, 1, 4, 6, 9]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
[+] Solve #0 -> Objective: 516.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [2, 6, 8, 10, 11, 12]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
[+] Solve #0 -> Objective: 560.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [2, 5, 10, 13, 15]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
[+] Solve #0 -> Objective: 529.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [3, 7, 8, 12]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
[+] Solve #0 -> Objective: 530.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [0, 4, 6, 8, 12]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
[+] Solve #0 -> Objective: 500.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices)

In [4]:
model.solve_times

[3.186079502105713]

# Application to GIFT-32

In [5]:
for idx in range(10):
    model = GenCQMModel(
        solver_name="hybrid_binary_quadratic_model_version2p",
        label="gift32-cqm2bqm",
        bqm_lagrange=15.0
    )

    SBOX_SIZE = 4
    N_SBOXES = 8
    BLOCKSIZE = SBOX_SIZE * N_SBOXES
    BITPERM = GIFT32_BITPERM
    assert len(BITPERM) == BLOCKSIZE
    assert set(BITPERM) == set(range(BLOCKSIZE))

    rounds = []
    n_rounds = 8

    for r in range(n_rounds):
        S_in, S_out = model.add_round(sbox_size=SBOX_SIZE, n_sboxes=N_SBOXES)
        rounds.append((S_in, S_out))
        model.add_sbox_layer(S_in, S_out, sbox_constraints,
                             sbox_size=SBOX_SIZE, n_boxes=N_SBOXES)

    for r in range(n_rounds - 1):
        model.connect_rounds_with_permutation(rounds[r][1], rounds[r + 1][0], BITPERM)

    for i in range(BLOCKSIZE):
        model.cqm.add_constraint(rounds[0][0][i] == (1 if i >= 1 else 0), label=f"fix_in_{i}")

    model.set_objective([1] * BLOCKSIZE, rounds[n_rounds - 1][1], sense='min')

    model.solve_for_integral_distinguisher(
        rounds[n_rounds - 1][1],
        blocksize=BLOCKSIZE,
        verbose=True
    )
    Hbqm_v2p_32_time.append(sum(model.solve_times))
print(Hbqm_v2p_32_time)

[+] Solve #0 -> Objective: 2039.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [0, 1, 2, 4, 8, 9, 13, 18, 22, 23, 24, 28, 29, 31]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
[+] Solve #0 -> Objective: 2018.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [6, 7, 11, 13, 19, 20, 24, 28]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
[+] Solve #0 -> Objective: 1868.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [2, 4, 7, 10, 11, 16, 22, 25]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
[+] Solve #0 -> Objective: 1914.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [2, 4, 10, 13, 16, 21, 26, 27, 28]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
[+] Solve #0 -> Objective: 2034.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [0, 4, 8, 12, 19, 20, 24, 26, 28]

Zero-forced bits:
  (none — distinguisher confirmed without for

# Application to GIFT-64

In [6]:
for idx in range(10):
    model = GenCQMModel(
        solver_name="hybrid_binary_quadratic_model_version2p", 
        label="gift16-cqm2bqm",
        bqm_lagrange=15.0
    )

    SBOX_SIZE = 4
    N_SBOXES = 16
    BLOCKSIZE = SBOX_SIZE * N_SBOXES
    BITPERM = GIFT64_BITPERM
    assert len(BITPERM) == BLOCKSIZE
    assert set(BITPERM) == set(range(BLOCKSIZE))

    rounds = []
    n_rounds = 9

    for r in range(n_rounds):
        S_in, S_out = model.add_round(sbox_size=SBOX_SIZE, n_sboxes=N_SBOXES)
        rounds.append((S_in, S_out))
        model.add_sbox_layer(S_in, S_out, sbox_constraints,
                             sbox_size=SBOX_SIZE, n_boxes=N_SBOXES)

    for r in range(n_rounds - 1):
        model.connect_rounds_with_permutation(rounds[r][1], rounds[r + 1][0], BITPERM)

    for i in range(BLOCKSIZE):
        model.cqm.add_constraint(rounds[0][0][i] == (1 if i >= 1 else 0), label=f"fix_in_{i}")

    model.set_objective([1] * BLOCKSIZE, rounds[n_rounds - 1][1], sense='min')

    model.solve_for_integral_distinguisher(
        rounds[n_rounds - 1][1],
        blocksize=BLOCKSIZE,
        verbose=True
    )
    Hbqm_v2p_64_time.append(sum(model.solve_times))
print(Hbqm_v2p_64_time)

[+] Solve #0 -> Objective: 4644.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [2, 3, 4, 5, 9, 12, 13, 14, 15, 19, 23, 28, 38, 39, 40, 41, 46, 47, 50, 51, 54, 56, 59, 60]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
[+] Solve #0 -> Objective: 4712.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [3, 5, 11, 12, 13, 16, 22, 23, 26, 35, 38, 40, 45, 48, 55, 57, 62]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
[+] Solve #0 -> Objective: 4608.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [0, 6, 14, 16, 20, 24, 26, 27, 30, 32, 39, 40, 44, 46, 50, 56, 60, 63]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
[+] Solve #0 -> Objective: 4820.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [0, 10, 11, 12, 15, 18, 22, 24, 30, 34, 37, 42, 43, 45, 46, 49, 53, 54, 56, 61]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
[+] Solve #0 -> Objective: 476

# Application to GIFT-128

In [ ]:
for idx in range(10):
    model = GenCQMModel(
        solver_name="hybrid_binary_quadratic_model_version2p",
        label="gift16-cqm2bqm",
        bqm_lagrange=15.0
    )

    SBOX_SIZE = 4
    N_SBOXES = 32
    BLOCKSIZE = SBOX_SIZE * N_SBOXES
    BITPERM = GIFT128_BITPERM
    assert len(BITPERM) == BLOCKSIZE
    assert set(BITPERM) == set(range(BLOCKSIZE))

    rounds = []
    n_rounds = 10

    for r in range(n_rounds):
        S_in, S_out = model.add_round(sbox_size=SBOX_SIZE, n_sboxes=N_SBOXES)
        rounds.append((S_in, S_out))
        model.add_sbox_layer(S_in, S_out, sbox_constraints,
                             sbox_size=SBOX_SIZE, n_boxes=N_SBOXES)

    for r in range(n_rounds - 1):
        model.connect_rounds_with_permutation(rounds[r][1], rounds[r + 1][0], BITPERM)

    for i in range(BLOCKSIZE):
        model.cqm.add_constraint(rounds[0][0][i] == (1 if i >= 1 else 0), label=f"fix_in_{i}")

    model.set_objective([1] * BLOCKSIZE, rounds[n_rounds - 1][1], sense='min')

    model.solve_for_integral_distinguisher(
        rounds[n_rounds - 1][1],
        blocksize=BLOCKSIZE,
        verbose=True
    )
    Hbqm_v2p_128_time.append(sum(model.solve_times))
print(Hbqm_v2p_128_time)

[+] Solve #0 -> Objective: 10600.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [2, 6, 7, 8, 12, 19, 22, 23, 26, 29, 33, 39, 42, 43, 44, 49, 50, 54, 58, 62, 64, 66, 74, 78, 82, 84, 88, 93, 94, 97, 99, 102, 103, 104, 105, 111, 112, 115, 116, 125]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
[+] Solve #0 -> Objective: 10702.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [0, 6, 8, 14, 16, 17, 25, 27, 33, 38, 43, 46, 48, 49, 51, 60, 67, 70, 71, 75, 78, 79, 87, 88, 92, 96, 98, 99, 103, 104, 111, 112, 118, 121, 123, 124, 125]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)


# Advantage_system6.4

# Application to GIFT-16

In [3]:
for idx in range(10):
    model = GenCQMModel(
        solver_name="Advantage_system6.4",
        label="gift16-cqm2bqm",
        bqm_lagrange=15.0
    )

    SBOX_SIZE = 4
    N_SBOXES = 4
    BLOCKSIZE = SBOX_SIZE * N_SBOXES
    BITPERM = GIFT16_BITPERM
    assert len(BITPERM) == BLOCKSIZE
    assert set(BITPERM) == set(range(BLOCKSIZE))

    rounds = []
    n_rounds = 5

    for r in range(n_rounds):
        S_in, S_out = model.add_round(sbox_size=SBOX_SIZE, n_sboxes=N_SBOXES)
        rounds.append((S_in, S_out))
        model.add_sbox_layer(S_in, S_out, sbox_constraints,
                             sbox_size=SBOX_SIZE, n_boxes=N_SBOXES)

    for r in range(n_rounds - 1):
        model.connect_rounds_with_permutation(rounds[r][1], rounds[r + 1][0], BITPERM)

    for i in range(BLOCKSIZE):
        model.cqm.add_constraint(rounds[0][0][i] == (1 if i >= 1 else 0), label=f"fix_in_{i}")

    model.set_objective([1] * BLOCKSIZE, rounds[n_rounds - 1][1], sense='min')

    model.solve_for_integral_distinguisher(
        rounds[n_rounds - 1][1],
        blocksize=BLOCKSIZE,
        verbose=True
    )
    Advantage_6_4_16_time.append(sum(model.solve_times))
print(Advantage_6_4_16_time)

[+] Solve #0 -> Objective: 513.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [3, 7, 8]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
[+] Solve #0 -> Objective: 575.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [2, 3, 4, 8, 13]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
[+] Solve #0 -> Objective: 558.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [2, 8, 14]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
[+] Solve #0 -> Objective: 514.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [1, 4, 8, 15]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
[+] Solve #0 -> Objective: 500.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [2, 5, 7, 8, 14]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
[+] Solve #0 -> Objective: 528.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [8, 14, 15]

Zero

# Application to GIFT-32

In [4]:
for idx in range(10):
    model = GenCQMModel(
        solver_name="Advantage_system6.4",
        label="gift32-cqm2bqm",
        bqm_lagrange=15.0
    )

    SBOX_SIZE = 4
    N_SBOXES = 8
    BLOCKSIZE = SBOX_SIZE * N_SBOXES
    BITPERM = GIFT32_BITPERM
    assert len(BITPERM) == BLOCKSIZE
    assert set(BITPERM) == set(range(BLOCKSIZE))

    rounds = []
    n_rounds = 8

    for r in range(n_rounds):
        S_in, S_out = model.add_round(sbox_size=SBOX_SIZE, n_sboxes=N_SBOXES)
        rounds.append((S_in, S_out))
        model.add_sbox_layer(S_in, S_out, sbox_constraints,
                             sbox_size=SBOX_SIZE, n_boxes=N_SBOXES)

    for r in range(n_rounds - 1):
        model.connect_rounds_with_permutation(rounds[r][1], rounds[r + 1][0], BITPERM)

    for i in range(BLOCKSIZE):
        model.cqm.add_constraint(rounds[0][0][i] == (1 if i >= 1 else 0), label=f"fix_in_{i}")

    model.set_objective([1] * BLOCKSIZE, rounds[n_rounds - 1][1], sense='min')

    model.solve_for_integral_distinguisher(
        rounds[n_rounds - 1][1],
        blocksize=BLOCKSIZE,
        verbose=True
    )
    Advantage_6_4_32_time.append(sum(model.solve_times))
print(Advantage_6_4_32_time)

[+] Solve #0 -> Objective: 1990.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [0, 9, 10, 14, 18, 20, 21, 26, 29, 31]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
[+] Solve #0 -> Objective: 2021.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [0, 4, 6, 10, 12, 15, 17, 21, 24, 26, 28]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
[+] Solve #0 -> Objective: 2020.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [0, 4, 6, 10, 15, 16, 22, 24, 26, 28]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
[+] Solve #0 -> Objective: 2004.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [1, 10, 11, 12, 16, 20, 23, 24, 30]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
[+] Solve #0 -> Objective: 2005.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [0, 4, 11, 14, 16, 24, 26, 27, 30, 31]

Zero-forced bits:
  (none — distinguisher confirme

# Application to GIFT-64

In [3]:
for idx in range(10):
    model = GenCQMModel(
        solver_name="Advantage_system6.4",
        label="gift64-cqm2bqm",
        bqm_lagrange=15.0
    )

    SBOX_SIZE = 4
    N_SBOXES = 16
    BLOCKSIZE = SBOX_SIZE * N_SBOXES
    BITPERM = GIFT64_BITPERM
    assert len(BITPERM) == BLOCKSIZE
    assert set(BITPERM) == set(range(BLOCKSIZE))

    rounds = []
    n_rounds = 9

    for r in range(n_rounds):
        S_in, S_out = model.add_round(sbox_size=SBOX_SIZE, n_sboxes=N_SBOXES)
        rounds.append((S_in, S_out))
        model.add_sbox_layer(S_in, S_out, sbox_constraints,
                             sbox_size=SBOX_SIZE, n_boxes=N_SBOXES)

    for r in range(n_rounds - 1):
        model.connect_rounds_with_permutation(rounds[r][1], rounds[r + 1][0], BITPERM)

    for i in range(BLOCKSIZE):
        model.cqm.add_constraint(rounds[0][0][i] == (1 if i >= 1 else 0), label=f"fix_in_{i}")

    model.set_objective([1] * BLOCKSIZE, rounds[n_rounds - 1][1], sense='min')

    model.solve_for_integral_distinguisher(
        rounds[n_rounds - 1][1],
        blocksize=BLOCKSIZE,
        verbose=True
    )
    print(sum(model.solve_times))
    Advantage_6_4_64_time.append(sum(model.solve_times))
print(Advantage_6_4_64_time)

[+] Solve #0 -> Objective: 4744.00

[✓] Integral Distinguisher FOUND!
Integral bits (indices): [2, 8, 9, 12, 15, 16, 20, 23, 26, 30, 31, 32, 33, 40, 47, 50, 52, 56, 57]

Zero-forced bits:
  (none — distinguisher confirmed without forcing)
8.525911569595337


SolverFailureError: Problem not accepted because user has insufficient remaining solver access time in project 73xL